In [ ]:
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
!pip install utils
from utils import *
import random
from spacy.tokenizer import Tokenizer
import warnings
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
warnings.filterwarnings('ignore') 
import string
from sklearn.pipeline import make_pipeline
import xlsxwriter 
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import time
from spacy.lang.tr import Turkish
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score
from transformers import AutoModel, AutoTokenizer
from glove import Corpus, Glove
import fasttext
import fasttext.util
import nltk 
import torch

In [ ]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    
    return accuracy_score(y_true, y_pred) # return accuracy score

In [ ]:
def classification_report_with_precision_score(y_true, y_pred):
    
    return precision_score(y_true, y_pred, average='weighted')

In [ ]:
def classification_report_with_recall_score(y_true, y_pred):
    
    return recall_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_f1_score(y_true, y_pred):
    
    return f1_score(y_true, y_pred,average='weighted')

In [ ]:
X_t = [word_tokenize(i) for i in df.tweets.tolist()]

In [ ]:
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(X_t, window=5)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=32, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=50, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

In [ ]:
wordembeddings=glove.load('glove.model')

In [ ]:
def feature_extraction_glove(text):
    sentence=""
    sentence=str(text) #alınan veriyi string çeviriyoruz
    tokens=word_tokenize(sentence)
    new_tokens=[token for token in tokens] #stopword listesinde olan kelimeleri çıkarıyoruz
    sent_list=[]

    for word in new_tokens:
        wv=wordembeddings.word_vectors[wordembeddings.dictionary[word]] #glove modeline kelimeyi yolluyorum
        sent_list.append(wv)
 
    sent_embed= np.mean(sent_list,axis=0) # fasttext gösterimlerinin ortalamasını alarak cümle vektörünü çıktı olarak veriyor
    return sent_embed


In [ ]:
y_total= []
y_total=df.duygu.tolist()


X_total = []
for element in tqdm(df.tweets):
    X_total.append(feature_extraction_glove(element))


In [ ]:
# Nested CV with parameter optimization
clf = svm.SVC(kernel='linear')

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for glove/SVM: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for glove/SVM: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for glove/SVM: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for glove/SVM: ",f1.mean())

In [ ]:
# Nested CV with parameter optimization
clf = LogisticRegression()

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for glove/LogisticRegression: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for glove/LogisticRegression: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for glove/LogisticRegression: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for glove/LogisticRegression: ",f1.mean())


In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10, 3), random_state=1)

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for glove/MLPC: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for glove/MLPC: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for glove/MLPC: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for glove/MLPC: ",f1.mean())
